In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/final_df_complete.csv")


In [3]:
# The goal of this exercise will be, for every Constructor, to modelate 0s for repeated Drivers after every Season and 1s for different ones. Thus we will get a binary idea of how many times a Team switched Drivers between Seasons.

# Step one, pulverize the problem into various subsets of things. Let's only get the Constructor, their Season, and with which Driver they competed. 

constructors = dataset.groupby('constructor').apply(lambda a: a.drop('constructor', axis=1)[:])

df = constructors.reset_index().drop(['level_1'], axis=1)

driver_change_dataset = df.sort_values(by=['constructor','driver']).reset_index().drop(['index'], axis=1)

driver_change_dataset.head(10)

,constructor,season,round,circuit_id,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,...,driver_standings_pos_after_race.1,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age,wins_percentage
0,ags,1990,2,interlagos,True,False,False,False,False,dalmas,...,0,0,0,0,0,0,17,3.810,28,0.0
1,ags,1990,14,jerez,True,False,False,False,False,dalmas,...,0,0,0,0,0,0,16,4.329,29,0.0
2,ags,1987,1,jacarepagua,True,False,False,False,False,fabre,...,0,0,0,0,0,0,0,13.688,27,0.0
3,ags,1987,2,imola,True,False,False,False,False,fabre,...,12,0,0,0,0,0,8,8.046,27,0.0
4,ags,1987,3,spa,False,True,False,False,True,fabre,...,17,0,0,12,0,0,11,15.335,27,0.0
5,ags,1987,4,monaco,True,False,False,False,False,fabre,...,19,0,0,17,0,0,12,7.766,27,0.0
6,ags,1987,5,detroit,True,False,False,True,False,fabre,...,22,0,0,19,0,0,13,14.380,27,NaN
7,ags,1987,6,ricard,True,False,False,False,False,fabre,...,22,0,0,22,0,0,13,8.245,27,0.0
8,ags,1987,7,silverstone,True,False,False,False,False,fabre,...,22,0,0,22,0,0,13,9.582,27,0.0
9,ags,1987,8,hockenheimring,False,False,False,False,False,fabre,...,22,0,0,22,0,0,13,12.381,27,0.0


In [4]:
# Now here's the trick, we will use a where clause to codify our data.

driver_change_dataset['change_drivers'] = np.where((driver_change_dataset['constructor'] == driver_change_dataset['constructor'].shift()) & (driver_change_dataset['driver'] == driver_change_dataset['driver'].shift()),0,1)


In [5]:
# Let's check Mclaren's career again, we should see the correct codification now!

driver_change_dataset[['constructor','season','driver','change_drivers']][(driver_change_dataset['constructor']=='mclaren')].head(90)

,constructor,season,driver,change_drivers
5965,mclaren,2007,alonso,1
5966,mclaren,2007,alonso,0
5967,mclaren,2007,alonso,0
5968,mclaren,2007,alonso,0
5969,mclaren,2007,alonso,0
...,...,...,...,...
6050,mclaren,2018,alonso,0
6051,mclaren,2018,alonso,0
6052,mclaren,2018,alonso,0
6053,mclaren,2018,alonso,0


In [6]:
# Now let's sort things out in order to be able to use the dataset as a time series.

driver_change_dataset = driver_change_dataset.sort_values(by=['season','round','constructor','driver']).reset_index().drop(['index'], axis=1)


In [7]:
driver_change_dataset.to_csv(r'/Users/alejandropalacios/Desktop/Ironhack/Data Analytics Bootcamp/Advanced Data Analysis Techniques/Project/F1-Grand-Prix-Predictor/Data Manipulation/Modeling/driver_change_dataset.csv', index = False)